In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Runnables

## Métodos dos Runnables de LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream:** transmitir de volta fragmentos da resposta
- **invoke:** chamar a cadeia com um input
- **batch:** chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- **astream:** transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke:** chamar a cadeia com um input de forma assíncrona
- **abatch:** chamar a cadeia com uma lista de inputs de forma assíncrona

In [3]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate as cpt

model = ChatOpenAI()
prompt = cpt.from_template('Crie uma frase sobre o assunto: {assunto}')

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta

In [4]:
chain.invoke({'assunto': 'Mudanças Climáticas'})

AIMessage(content='"As mudanças climáticas são um alerta urgente para a necessidade de agir de forma sustentável e responsável com o planeta."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 21, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d7cbf914-8b10-4ef4-94d2-4ff0df98bad8-0', usage_metadata={'input_tokens': 21, 'output_tokens': 32, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário.

In [5]:
chain.invoke('Mudanças Climáticas')

AIMessage(content='As mudanças climáticas são um alerta para a necessidade urgente de agirmos em prol da preservação do nosso planeta.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 21, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-abf7c254-532f-457d-a4de-c455252559f1-0', usage_metadata={'input_tokens': 21, 'output_tokens': 31, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream


In [6]:
for stream in chain.stream('Computador Quântico'):
    print(stream.content, end='')

O computador quântico revolucionará a forma como lidamos com a computação, abrindo portas para novas descobertas e avanços tecnológicos incríveis.

## Batch

Para fazermos múltimpas requisições em paralelo utilizamos o batch

In [9]:
chain.batch([{'assunto': 'gatinhos'}, {'assunto': 'cachorrinhos'}, {'assunto': 'papagaios'}])

[AIMessage(content='Os gatinhos são seres adoráveis e cheios de charme, capazes de conquistar nossos corações com apenas um olhar.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 19, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0993076d-795d-4c20-a465-68949987a97f-0', usage_metadata={'input_tokens': 19, 'output_tokens': 32, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Cachorrinhos são a alegria em quatro patas que transformam nossos dias em pura felicidade.', additional_kwargs={'refusal': None}, response_metadata={'token_usa

In [10]:
chain.batch([{'assunto': 'gatinhos'}, {'assunto': 'cachorrinhos'}, {'assunto': 'papagaios'}], config={'max_concurrency': 2})

[AIMessage(content='Gatinhos são seres adoráveis que nos enchem de amor e alegria com suas travessuras e ronronados.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 19, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ed9fefc5-8616-4a3f-b67e-81408ed2ae0b-0', usage_metadata={'input_tokens': 19, 'output_tokens': 30, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Cachorrinhos são seres cheios de amor e alegria, que nos ensinam o verdadeiro significado da lealdade e companheirismo.', additional_kwargs={'refusal': None}, response_metad

## Runnables especiais

### Rodando em paralelo
```
     Input      
      / \       
     /   \      
 Branch1 Branch2
     \   /      
      \ /       
      Combine   
```

In [16]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI()
prompt = cpt.from_template('Crie um nome para o seguinte produto: {produto}')

chain_nome = prompt | model | StrOutputParser()

In [17]:
prompt = cpt.from_template('Descreva o cliente potencial para o seguinte produto: {produto}')

chain_clientes = prompt | model | StrOutputParser()

In [18]:
prompt = cpt.from_template("""Dado o produto com o seguinte nome e seguinte público alvo, desenvolva um anúncio para o produto.
                           
Nome do produto: {nome_produto}
Público: {publico}""")

In [19]:
paralel = RunnableParallel({'nome_produto': chain_nome, 'publico': chain_clientes})
paralel.invoke({'produto': 'Automatizar processos usando inteligência artificial'})

{'nome_produto': 'AutomatIA - Automatização Inteligente',
 'publico': 'O cliente potencial para um produto que automatiza processos utilizando inteligência artificial seria uma empresa que busca aumentar a eficiência e a produtividade de suas operações. Este cliente provavelmente possui uma grande quantidade de tarefas repetitivas e demoradas que podem ser otimizadas através da automação, como por exemplo, o processamento de dados, a análise de informações ou a gestão de sistemas.\n\nAlém disso, o cliente potencial para este produto provavelmente valoriza a inovação e está disposto a investir em tecnologias avançadas para se manter competitivo no mercado. Também é possível que já tenha alguma experiência com inteligência artificial e esteja em busca de soluções mais sofisticadas e personalizadas para atender às suas necessidades específicas.\n\nEm resumo, o cliente potencial para um produto de automação com inteligência artificial é uma empresa que busca melhorar a eficiência de suas o

In [20]:
chain = paralel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Automatizar processos usando inteligência artificial'})

'🚀 AutomAIzador de Processos: A solução para otimizar a sua empresa!\n\nSe a sua empresa busca aumentar a eficiência, produtividade e qualidade das operações, o AutomAIzador de Processos é a solução perfeita para você! Com a nossa tecnologia de ponta de inteligência artificial, você poderá automatizar processos manuais e repetitivos, reduzir erros, minimizar retrabalho e garantir a consistência em todas as suas atividades.\n\nNão perca mais tempo e recursos com tarefas que podem ser automatizadas! Invista no AutomAIzador de Processos e leve a sua empresa para o próximo nível de competitividade e inovação. Entre em contato conosco e descubra como podemos transformar o seu negócio com tecnologia disruptiva. Automatize, otimize e cresça com o AutomAIzador de Processos! 🤖✨'

In [21]:
from langchain_core.runnables import RunnableLambda

def cumprimentar(nome):
    return f'Olá, {nome}!'

runnable_cumprimentar = RunnableLambda(cumprimentar)

resultado = runnable_cumprimentar.invoke('Maria')
print(resultado)

Olá, Maria!


In [27]:
from langchain_core.runnables import RunnablePassthrough

prompt = cpt.from_template("""Dado o produto com o seguinte nome e o seguinte público potencial, desenvolva um anúncio para o produto.
                           
Nome do produto: {nome_produto}
Característica do produto: {produto}
Público: {publico}""")

parallel = RunnablePassthrough().assign(**{'nome_produto': chain_nome, 'publico': chain_clientes})
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Automatizar processos usando inteligência artificial'})

'\n\nAnúncio:\n\nVocê está cansado de gastar horas em tarefas repetitivas e demoradas? Conheça o AutomateAI, a solução perfeita para simplificar seus processos com inteligência artificial!\n\nCom o AutomateAI, você poderá automatizar tarefas tediosas e aumentar a eficiência de suas operações, economizando tempo e recursos preciosos. Seja na análise de dados, no gerenciamento de projetos ou na otimização de processos, nossa tecnologia de ponta está pronta para revolucionar a forma como você trabalha.\n\nNão fique para trás no mercado competitivo de hoje. Adote o AutomateAI e mantenha sua empresa à frente das tendências tecnológicas, garantindo uma vantagem estratégica e um aumento significativo na produtividade.\n\nEntre em contato conosco hoje mesmo e descubra como o AutomateAI pode transformar o seu negócio!'